In [1]:
# %%
"""
Tests metrics betwen stochastic process neural representations.
"""
import numpy as np
from netrep.metrics import GPStochasticMetric,GaussianStochasticMetric,GPStochasticDiff
from netrep.utils import rand_orth
from sklearn.utils.validation import check_random_state
from sklearn.covariance import EmpiricalCovariance

from numpy import random as rand
from netrep.utils import rand_orth

%load_ext autoreload
%autoreload 2

/mnt/home/anejatbakhsh/anaconda3/envs/netrep/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# %% Class for sampling from a gaussian process given a kernel
class GaussianProcess:
    def __init__(self,kernel,D):
        self.kernel = kernel
        self.D = D

    def evaluate_kernel(self, xs, ys):
        fun = np.vectorize(self.kernel)
        return fun(xs[:, None], ys)

    def sample(self,ts,seed=0):
        np.random.seed(seed)

        T = ts.shape[0]
        c_g = self.evaluate_kernel(ts,ts)
        fs = rand.multivariate_normal(
            mean=np.zeros(T),
            cov=c_g,
            size=self.D
        )
        return fs

In [10]:
seed = 0
t = 5
n = 2
k = 100

# Set random seed, draw random rotation
rs = check_random_state(seed)
if n > 1:
    Q = rand_orth(n, n, random_state=rs)
else:
    Q = 1
    
# Generate data from a gaussian process with RBF kernel
ts = np.linspace(0,1,t)
gpA = GaussianProcess(
    kernel = lambda x, y: 1e-2*(1e-6*(x==y)+np.exp(-np.linalg.norm(x-y)**2/(2*1.**2))),
    D=n
)
sA = np.array([gpA.sample(ts,seed=i) for i in range(k)]).reshape(k,n*t)

# Transform GP according to a rotation applied to individiual 
# blocks of the full covariance matrix
A = [sA.mean(0),EmpiricalCovariance().fit(sA).covariance_]
B = [
    np.kron(np.eye(t),Q)@A[0],
    np.kron(np.eye(t),Q)@A[1]@(np.kron(np.eye(t),Q)).T
]

In [14]:
# Using alternating optimization and Orthogonal Procrustes
# Compute dSSD
metric = GPStochasticMetric(n_dims=n,group="orth")
dssd = metric.fit_score(A,B)

# Compute aSSD
metric = GPStochasticMetric(
    n_dims=n,
    group="orth",
    type='adapted',
)
assd = metric.fit_score(A,B)

# Compute mSSD
metric = GaussianStochasticMetric(group="orth")
A_marginal = [
    A[0].reshape(t,n),
    np.array([A[1][i*n:(i+1)*n,i*n:(i+1)*n] for i in range(t)])
]
B_marginal = [
    B[0].reshape(t,n),
    np.array([B[1][i*n:(i+1)*n,i*n:(i+1)*n] for i in range(t)])
]
mssd = metric.fit_score(A_marginal,B_marginal)

print('DSSD: ', dssd, ', Marginal SSD: ', mssd, ', Adapted SSD: ', assd)

DSSD:  -7.450580596923828e-09 , Marginal SSD:  0.0 , Adapted SSD:  9.125060374972147e-09


In [13]:
# Using differentiable optimization and Cayley orthogonal parameterization

metric = GPStochasticDiff(n_dims=n,n_times=t,type="Bures")
dssd = metric.fit_score(A,B,lr=1e-3,tol=1e-5,epsilon=1e-6)

metric = GPStochasticDiff(n_dims=n,n_times=t,type="Adapted_Bures")
assd = metric.fit_score(A,B,lr=1e-3,tol=1e-5,epsilon=1e-6)

metric = GPStochasticDiff(n_dims=n,n_times=t,type="Knothe_Rosenblatt")
kssd = metric.fit_score(A,B,lr=1e-3,tol=1e-5,epsilon=1e-6)

metric = GPStochasticDiff(n_dims=n,n_times=t,type="Marginal_Bures")
mssd = metric.fit_score(A,B,lr=1e-3,tol=1e-5,epsilon=1e-6)

print('DSSD: ', dssd, ', Adapted DSSD: ', assd, ', Marginal SSD: ', mssd, ', Knothe Rosenblatt SSD: ', kssd)

Iteration 700, loss 0.04: : 0it [00:01, ?it/s]
Iteration 200, loss 0.00: : 0it [00:00, ?it/s]
Iteration 200, loss 0.00: : 0it [00:00, ?it/s]
Iteration 700, loss 0.01: : 0it [00:02, ?it/s]

DSSD:  0.03943214 , Adapted DSSD:  0.0023483392 , Marginal SSD:  0.005165683 , Knothe Rosenblatt SSD:  0.0023267935
